<a href="https://colab.research.google.com/github/AJamal27891/AJamal/blob/master/License_Plate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://)

1.   Ahmed Gamal: 319730618
2.   Zara Akhter: 319712229



Main Steps



*  Detect car in input image using Yolo. In this method we had no struggle since we imported Darknet API. However, in order to train on a specific class, it gave a lot of errors.
*   Detect the plate within the car’s contour area.
*   Crop the license plate.
*   We applied two methods:

1.   Train a Neural Network (NN) on eMinst dataset.
2.   Train a Convolutional Neural Network (CNN) on the plate regions

*   Identify each character and calculate the loss
*   An additional method to calculate based on the 7-character combinations.

Datasets 

*   EMNIST Balanced Dataset:
1.  131,600 characters.
2.  47 balanced classes. 
*   License plate Dataset

We trained our CNN eMNIST dataset since it includes alphabets along with the numerical digits. Our CNN consisted of:

*    2 convolution layers.
*    2 maxpooling layers.
*    2 linear layers.
*    Optimizer Adam to find the best gradient.

We also added dropout layer as regularization since our model was overfitting with 99% accuracy on the training set. The accuracy reduced to 98% with the dropout regularization.

Additionally, we applied GridSearch to extract the best parameters for our model which were:

*    Learning Rate = 0.006
*    Max epochs = 10

This resulted in an accuracy of 92

Final Archetechre 2 layers of convolution 1 layer of Maxpooling three times then big flattend dense layer then 7 paralel dense layers every layer predict charchter. 
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_2 (InputLayer)            (None, 218, 1025, 3) 0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 216, 1023, 8) 224         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 214, 1021, 8) 584         conv2d_7[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 107, 510, 8)  0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 105, 508, 16) 1168        max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 103, 506, 16) 2320        conv2d_9[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 51, 253, 16)  0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 49, 251, 16)  2320        max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 47, 249, 16)  2320        conv2d_11[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 23, 124, 16)  0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 45632)        0           max_pooling2d_6[0][0]            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 64)           2920512     flatten_2[0][0]                  
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 33)           2145        dense_9[0][0]                    
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 33)           2145        dense_9[0][0]                    
==================================================================================




In [0]:
#@title Import libraries { display-mode: "form" }
from torchvision.datasets import EMNIST
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import gzip
import os 
import struct
import pandas as pd
from torchvision import transforms
import random
from imutils import perspective
from PIL import Image, ImageFilter
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from PIL import ImageOps
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [0]:
#@title GPU Code
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
#@title TPU code
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


In [0]:
import collections
from datetime import datetime, timedelta
import os
import requests
import threading

_VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
VERSION = "torch_xla==nightly"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
CONFIG = {
    'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
    'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
        (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
}[VERSION]
DIST_BUCKET = 'gs://tpu-pytorch/wheels'
TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# Update TPU XRT version
def update_server_xrt():
  print('Updating server-side XRT to {} ...'.format(CONFIG.server))
  url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
      TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
      XRT_VERSION=CONFIG.server,
  )
  print('Done updating server-side XRT: {}'.format(requests.post(url)))

update = threading.Thread(target=update_server_xrt)
update.start()

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install "$TORCH_WHEEL"
!pip install "$TORCH_XLA_WHEEL"
!pip install "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5
update.join()

In [0]:
#@title Connect to Google { display-mode: "form" }
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks'


In [0]:
#@title Class for transforming the data into torch dataset
from torch.utils.data import  Dataset, DataLoader
from torch.autograd import  Variable
class Train(Dataset):
  def __init__(self,transform=None):
    xy=torch.tensor(pd.read_csv('gdrive/My Drive/Colab Notebooks/emnist-balanced-train.csv').values)
    self.len = xy.shape[0]
    self.train_x = xy[:,1:].view(112799,1,28,28)
    self.train_y = xy[:, 0:1].view(1,112799)
    self.transform = transform
  def __getitem__ (self,index):
    if self.transform:
      train_x = self.transform 
    return self.train_x[index][-1],self.train_y[index][-1]
  def __len__(self):
    return self.len

class Test(Dataset):
  def __init__(self):
    xy=np.loadtxt('gdrive/My Drive/Colab Notebooks/emnist-balanced-test.csv',delimiter=',',dtype=np.float32)
    self.len = xy.shape[0]
    self.test_x = torch.from_numpy(xy[:,-1:])
    self.test_y = torch.from_numpy(xy[:,[-1]])
  def __getitem__ (self,index):
    return self.test_x[index+1],self.test_y[index+1]
  def __len__(self):
    return self.len
    

In [0]:
#@title Custom Emnist
from torchvision.datasets import MNIST
from __future__ import print_function
import os
import shutil
import tempfile
import torch
from torchvision.datasets.folder import ImageFolder
from torchvision.datasets.utils import check_integrity, download_and_extract_archive, extract_archive, \
    verify_str_arg

ARCHIVE_DICT = {
    'train': {
        'url': 'http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar',
        'md5': '1d675b47d978889d74fa0da5fadfb00e',
    },
    'val': {
        'url': 'http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar',
        'md5': '29b22e2961454d5413ddabcf34fc5622',
    },
    'devkit': {
        'url': 'http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_devkit_t12.tar.gz',
        'md5': 'fa75699e90414af021442c21a62c3abf',
    }
}



class EMNIST(MNIST):
    """`EMNIST <https://www.westernsydney.edu.au/bens/home/reproducible_research/emnist>`_ Dataset.

    Args:
        root (string): Root directory of dataset where ``EMNIST/processed/training.pt``
            and  ``EMNIST/processed/test.pt`` exist.
        split (string): The dataset has 6 different splits: ``byclass``, ``bymerge``,
            ``balanced``, ``letters``, ``digits`` and ``mnist``. This argument specifies
            which one to use.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    # Updated URL from https://www.nist.gov/node/1298471/emnist-dataset since the
    # _official_ download link
    # https://cloudstor.aarnet.edu.au/plus/s/ZNmuFiuQTqZlu9W/download
    # is (currently) unavailable
    url = 'https://cloudstor.aarnet.edu.au/plus/s/ZNmuFiuQTqZlu9W/download'
    md5 = "58c8d27c78d21e728a6bc7b3cc06412e"
    splits = ('byclass', 'bymerge', 'balanced', 'letters', 'digits', 'mnist')

    def __init__(self, root, split, **kwargs):
        self.split = verify_str_arg(split, "split", self.splits)
        self.training_file = self._training_file(split)
        self.test_file = self._test_file(split)
        super(EMNIST, self).__init__(root, **kwargs)

    @staticmethod
    def _training_file(split):
        return 'training_{}.pt'.format(split)

    @staticmethod
    def _test_file(split):
        return 'test_{}.pt'.format(split)

    def download(self):
        """Download the EMNIST data if it doesn't exist in processed_folder already."""
        import shutil

        if self._check_exists():
            return

        makedir_exist_ok(self.raw_folder)
        makedir_exist_ok(self.processed_folder)

        # download files
        print('Downloading and extracting zip archive')
        download_and_extract_archive(self.url, download_root=self.raw_folder, filename="emnist.zip",
                                     remove_finished=True, md5=self.md5)
        gzip_folder = os.path.join(self.raw_folder, 'gzip')
        for gzip_file in os.listdir(gzip_folder):
            if gzip_file.endswith('.gz'):
                extract_archive(os.path.join(gzip_folder, gzip_file), gzip_folder)

        # process and save as torch files
        for split in self.splits:
            print('Processing ' + split)
            training_set = (
                read_image_file(os.path.join(gzip_folder, 'emnist-{}-train-images-idx3-ubyte'.format(split))),
                read_label_file(os.path.join(gzip_folder, 'emnist-{}-train-labels-idx1-ubyte'.format(split)))
            )
            test_set = (
                read_image_file(os.path.join(gzip_folder, 'emnist-{}-test-images-idx3-ubyte'.format(split))),
                read_label_file(os.path.join(gzip_folder, 'emnist-{}-test-labels-idx1-ubyte'.format(split)))
            )
            with open(os.path.join(self.processed_folder, self._training_file(split)), 'wb') as f:
                torch.save(training_set, f)
            with open(os.path.join(self.processed_folder, self._test_file(split)), 'wb') as f:
                torch.save(test_set, f)
        shutil.rmtree(gzip_folder)

        print('Done!')


In [0]:
#@title extract ubyte method
from torchvision.datasets import DatasetFolder  
from torch.utils.data import DataLoader
from torchvision.datasets.vision import VisionDataset
from torchvision.datasets.folder import default_loader, make_dataset

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


class imgesfolder(Dataset):
  def __init__(self,root,transform=None,traget_transform=None,loader=default_loader):
    classes, class_to_idx = find_classes(root)
    imgs = make_dataset(root,class_to_idx,extensions='txt')
    if len(imgs) == 0:
      raise(RuntimeError('no files'))
    self.root = root
    self.imgs = imags
    self.class_to_idx = class_to_idx
    self.classes = classes 
    self.transform=transform
    self.traget_transform=target_transform
    self.loader=loader
  def __getitem__(self,index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is None:
      img = self.transform(img)
    if self.target_transform is None:
      target =self.target_transform(target)
    return img, target
  def __len__(self):
    return len(self.imgs)



In [0]:
#@title Custom Class for EMNIST
def makedir_exist_ok(dirpath):
    """
    Python2 support for os.makedirs(.., exist_ok=True)
    """
    try:
        os.makedirs(dirpath)
    except OSError as e:
        if e.errno == errno.EEXIST:
            pass
        else:
            raise

from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import gzip
import numpy as np
import torch
import codecs
import errno
import hashlib
from tqdm import tqdm

def read_image_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        parsed = np.frombuffer(data, dtype=np.uint8, offset=16)
        return torch.from_numpy(parsed).view(length, num_rows, num_cols)                

#%%
def read_label_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        parsed = np.frombuffer(data, dtype=np.uint8, offset=8)
        return torch.from_numpy(parsed).view(length).long()    

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)    

In [0]:
#@title DataLoader


train_set = EMNIST(root='EMNIST/processed/training.pt',split='balanced',train=True,download=True,transform = transforms.Compose([transforms.ToTensor()]))
test_set = EMNIST(root='EMNIST/processed/training.pt',split='balanced',train=False,download=True)

In [0]:
#@title Load the train and the test to the DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image
from PIL import ImageOps
from torchvision import datasets, transforms
transformation = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
train_data = Train(transform=transformation)
test_data = Test()
loader = DataLoader(train_data, batch_size=10)
test_loader = DataLoader(test_data,batch_size=10000,shuffle=True)

In [0]:
def get_all_preds(model,loader):
  all_preds= torch.tensor([])
  for batch in loader:
    images, labels = batch
    preds = model(images)
    all_preds = torch.cat((all_preds,preds),dim=0)

  return all_preds


In [0]:
#@title DataLoader
batch = 10000
train_loader = DataLoader(train_set,batch_size=batch,shuffle=True)
test_loader = DataLoader(test_set,batch_size=batch,shuffle=True)

In [0]:
#@title Building CNN arch
torch.set_grad_enabled(True)
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1 , out_channels=6 , kernel_size=5 )
    self.conv2 = nn.Conv2d(in_channels=6 , out_channels=12 , kernel_size=5 )


    self.fc1 =nn.Linear(in_features=12*4*4 , out_features= 120)
    self.fc2 = nn.Linear(in_features=120 , out_features=60 )
    self.out = nn.Linear(in_features=60 , out_features=47) 
    
  def forward(self,t):
    #input layer(1)
    
    t=t
    #hidden layer(2)
    t =self.conv1(t)
    t= F.relu(t)
    t= F.max_pool2d(t,kernel_size=2,stride=2)
    #hidden layer(3)
    t =self.conv2(t)
    t= F.relu(t)
    t= F.max_pool2d(t,kernel_size=2,stride=2)
    #hidden linear layer (4)
    t = t.reshape(-1,12*4*4)
    t= self.fc1(t)
    t= F.relu(t)
    #hidden linear layer (5)
    t= self.fc2(t)
    t= F.relu(t)
  
    #outputlayer
    t =self.out(t)
    t = F.softmax(t,dim=1)
    return t
net = Net()


In [0]:
sample = next(iter(train_set))
image,label = sample
image.shape
image= image.unsqueeze(0)
image.shape
pred = net(image)
print(pred)
print(label)
plt.imshow(image.view(28,28),cmap='gray')
plt.show()

In [0]:
optimizer = optim.Adam(net.parameters(),lr=0.01)

def get_num_correct(pred,labels):
        return pred.argmax(dim=1).eq(labels).sum().item()


accuracy_tabel = []

for epoch in range(50):
  total_loss = 0
  total_correct = 0

  for batch in train_loader:
    images,labels = batch 

    pred = net(images)

    loss = F.cross_entropy(pred,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    total_correct += get_num_correct(pred,labels)
    accuracy = 1-(total_loss/total_correct)
    accurcy_table = accuracy_tabel.append(accuracy)
  

  print('epoch',epoch,'total correct',total_correct, 'total loss', total_loss, 'accuracy ',accuracy )

accuracy_dict = { 'accuracy':[accuracy_tabel] }



In [0]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

new_net = NeuralNetClassifier(
    net,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

new_net.fit(images, labels)
y_proba = new_net.predict_proba(images)

params = {
    'lr': [0.01, 0.02,0.001],
    'max_epochs': [10, 20],
}
gs = GridSearchCV(new_net, params, refit=False, cv=30, scoring='accuracy')

gs.fit(np.array(images), np.array(labels))
print(gs.best_score_, gs.best_params_)

In [0]:
print(torchvision.__version__)

In [0]:
prediction_loader = torch.utils.data.DataLoader(train_set,batch_size=10000)
train_preds = get_all_preds(net,prediction_loader) 
train_preds.shape


In [0]:
print(train_preds.requires_grad)

In [0]:
with torch.no_grad():
  prediction_loader = torch.utils.data.DataLoader(train_set,batch_size=10000)
  train_preds = get_all_preds(net,prediction_loader) 


In [0]:
train_preds.shape

In [0]:
pred_correct = get_num_correct(train_preds,train_set.targets)
pred_correct

In [0]:
stacked = torch.stack((train_set.targets,train_preds.argmax(dim=1)),dim=1)
stacked

In [0]:
stacked[0].tolist()

In [0]:
cmt=torch.zeros(47,47,dtype= torch.int32)

In [0]:
for p in stacked:
  j,k = p.tolist()
  cmt[j,k]= cmt[j,k]+1

In [0]:
cmt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive//My\ Drive

In [0]:
import tensorflow as tf
import numpy as np
import time
import cv2
import os
import random


num_epochs = 300


INITIAL_LEARNING_RATE = 1e-3
DECAY_STEPS = 2000
LEARNING_RATE_DECAY_FACTOR = 0.9  # The learning rate decay factor
MOMENTUM = 0.9

REPORT_STEPS = 5000


BATCH_SIZE = 50
TRAIN_SIZE = 7368
BATCHES = TRAIN_SIZE//BATCH_SIZE
test_num = 3

ti = 'train'        
vi = 'valid'        
img_size = [94, 24]
tl = None
vl = None
num_channels = 3
label_len = 7

CHARS = [
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z','-','O','I'
         ]

CHARS_DICT = {char:i for i, char in enumerate(CHARS)}

NUM_CHARS = len(CHARS)


def encode_label(s):
    label = np.zeros([len(s)])
    for i, c in enumerate(s):
        label[i] = CHARS_DICT[c]
    return label

class TextImageGenerator:
    def __init__(self, img_dir, label_file, batch_size, img_size, num_channels=3, label_len=7):
        self._img_dir = img_dir
        self._label_file = label_file
        self._batch_size = batch_size
        self._num_channels = num_channels
        self._label_len = label_len
        self._img_w, self._img_h = img_size

        self._num_examples = 0
        self._next_index = 0
        self._num_epoches = 0
        self.filenames = []
        self.labels = []

        self.init()

    def init(self):
        self.labels = []
        fs = os.listdir(self._img_dir)
        for filename in fs:
                self.filenames.append(filename)
        for filename in self.filenames:
            label = filename[:7]
            
            label = encode_label(label)
            self.labels.append(label)
            self._num_examples += 1
        self.labels = np.float32(self.labels)

    def next_batch(self):
        # Shuffle the data
        if self._next_index == 0:
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._filenames = [self.filenames[i] for i in perm]
            self._labels = self.labels[perm]

        batch_size = self._batch_size
        start = self._next_index
        end = self._next_index + batch_size
        if end > self._num_examples:
            self._next_index = 0
            start = self._next_index
            end = self._next_index + batch_size
            self._num_epoches += 1
        else:
            self._next_index = end
        images = np.zeros([batch_size, self._img_h, self._img_w, self._num_channels])
        # labels = np.zeros([batch_size, self._label_len])

        for j, i in enumerate(range(start, end)):
            fname = self._filenames[i]
            img = cv2.imread(os.path.join(self._img_dir, fname))
            img = cv2.resize(img, (self._img_w, self._img_h), interpolation=cv2.INTER_CUBIC)
            images[j, ...] = img
        images = np.transpose(images, axes=[0, 2, 1, 3])
        labels = self._labels[start:end, ...]
        targets = [np.asarray(i) for i in labels]
        sparse_labels = sparse_tuple_from(targets)
        # input_length = np.zeros([batch_size, 1])

        seq_len = np.ones(self._batch_size) * 24
        return images, sparse_labels, seq_len

def sparse_tuple_from(sequences, dtype=np.int32):
    """
    Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape


def decode_sparse_tensor(sparse_tensor):
    decoded_indexes = list()
    current_i = 0
    current_seq = []
    for offset, i_and_index in enumerate(sparse_tensor[0]):
        i = i_and_index[0]
        if i != current_i:
            decoded_indexes.append(current_seq)
            current_i = i
            current_seq = list()
        current_seq.append(offset)
    decoded_indexes.append(current_seq)
    result = []
    for index in decoded_indexes:
        result.append(decode_a_seq(index, sparse_tensor))
    return result


def decode_a_seq(indexes, spars_tensor):
    decoded = []
    for m in indexes:
        str = CHARS[spars_tensor[1][m]]
        decoded.append(str)
    return decoded

def small_basic_block(x,im,om):
    x = conv(x,im,int(om/4),ksize=[1,1])
    x = tf.nn.relu(x)
    x = conv(x,int(om/4),int(om/4),ksize=[3,1],pad='SAME')
    x = tf.nn.relu(x)
    x = conv(x,int(om/4),int(om/4),ksize=[1,3],pad='SAME')
    x = tf.nn.relu(x)
    x = conv(x,int(om/4),om,ksize=[1,1])
    return x

def conv(x,im,om,ksize,stride=[1,1,1,1],pad = 'SAME'):
    conv_weights = tf.Variable(
        tf.truncated_normal([ksize[0], ksize[1], im, om],  # 5x5 filter, depth 32.
                            stddev=0.1,
                            seed=None, dtype=tf.float32))
    conv_biases = tf.Variable(tf.zeros([om], dtype=tf.float32))
    out = tf.nn.conv2d(x,
                        conv_weights,
                        strides=stride,
                        padding=pad)
    relu = tf.nn.bias_add(out, conv_biases)
    return relu

def get_train_model(num_channels, label_len, b, img_size):
    inputs = tf.placeholder(
        tf.float32,
        shape=(b, img_size[0], img_size[1], num_channels))

  #ctc_loss
    targets = tf.sparse_placeholder(tf.int32)

   #[batch_size,]
    seq_len = tf.placeholder(tf.int32, [None])
    x = inputs

    x = conv(x,num_channels,64,ksize=[3,3])
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x,
                          ksize=[1, 3, 3, 1],
                          strides=[1, 1, 1, 1],
                          padding='SAME')
    x = small_basic_block(x,64,64)
    x2=x
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)

    x = tf.nn.max_pool(x,
                          ksize=[1, 3, 3, 1],
                          strides=[1, 2, 1, 1],
                          padding='SAME')
    x = small_basic_block(x, 64,256)
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)
    x = small_basic_block(x, 256, 256)
    x3 = x
    x = tf.layers.batch_normalization(x)

    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x,
                       ksize=[1, 3, 3, 1],
                       strides=[1, 2, 1, 1],
                       padding='SAME')
    x = tf.layers.dropout(x)

    x = conv(x, 256, 256, ksize=[4, 1])
    x = tf.layers.dropout(x)
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)


    x = conv(x,256,NUM_CHARS+1,ksize=[1,13],pad='SAME')
    x = tf.nn.relu(x)
    cx = tf.reduce_mean(tf.square(x))
    x = tf.div(x,cx)

    #x = tf.reduce_mean(x,axis = 2)
    #x1 = conv(inputs,num_channels,num_channels,ksize = (5,1))


    x1 = tf.nn.avg_pool(inputs,
                       ksize=[1, 4, 1, 1],
                       strides=[1, 4, 1, 1],
                       padding='SAME')
    cx1 = tf.reduce_mean(tf.square(x1))
    x1 = tf.div(x1, cx1)

    # x1 = tf.image.resize_images(x1, size = [18, 16], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    x2 = tf.nn.avg_pool(x2,
                        ksize=[1, 4, 1, 1],
                        strides=[1, 4, 1, 1],
                        padding='SAME')
    cx2 = tf.reduce_mean(tf.square(x2))
    x2 = tf.div(x2, cx2)

    #x2 = tf.image.resize_images(x2, size=[18, 16], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    x3 = tf.nn.avg_pool(x3,
                        ksize=[1, 2, 1, 1],
                        strides=[1, 2, 1, 1],
                        padding='SAME')
    cx3 = tf.reduce_mean(tf.square(x3))
    x3 = tf.div(x3, cx3)

    #x3 = tf.image.resize_images(x3, size=[18, 16], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)


    #x1 = tf.nn.relu(x1)

    x = tf.concat([x,x1,x2,x3],3)
    x = conv(x, x.get_shape().as_list()[3], NUM_CHARS + 1, ksize=(1, 1))
    logits = tf.reduce_mean(x,axis=2)
    # x_shape = x.get_shape().as_list()
    # outputs = tf.reshape(x, [-1,x_shape[2]*x_shape[3]])
    # W1 = tf.Variable(tf.truncated_normal([x_shape[2]*x_shape[3],
    #                                      150],
    #                                     stddev=0.1))
    # b1 = tf.Variable(tf.constant(0., shape=[150]))
    # # [batch_size*max_timesteps,num_classes]
    # x = tf.matmul(outputs, W1) + b1
    # x= tf.layers.dropout(x)
    # x = tf.nn.relu(x)
    # W2 = tf.Variable(tf.truncated_normal([150,
    #                                      NUM_CHARS+1],
    #                                     stddev=0.1))
    # b2 = tf.Variable(tf.constant(0., shape=[NUM_CHARS+1]))
    # x = tf.matmul(x, W2) + b2
    # x = tf.layers.dropout(x)
    # # [batch_size,max_timesteps,num_classes]
    # logits = tf.reshape(x, [b, -1, NUM_CHARS+1])

    return logits, inputs, targets, seq_len

def train(a):

    train_gen = TextImageGenerator(img_dir=ti,
                                   label_file=tl,
                                   batch_size=BATCH_SIZE,
                                   img_size=img_size,
                                   num_channels=num_channels,
                                   label_len=label_len)

    val_gen = TextImageGenerator(img_dir=vi,
                                 label_file=vl,
                                 batch_size=BATCH_SIZE,
                                 img_size=img_size,
                                 num_channels=num_channels,
                                 label_len=label_len)
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                               global_step,
                                               DECAY_STEPS,
                                               LEARNING_RATE_DECAY_FACTOR,
                                               staircase=True)
    logits, inputs, targets, seq_len = get_train_model(num_channels, label_len,BATCH_SIZE, img_size)
    logits = tf.transpose(logits, (1, 0, 2))
    # tragets
    loss = tf.nn.ctc_loss(labels=targets, inputs=logits, sequence_length=seq_len)
    cost = tf.reduce_mean(loss)

    # optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=MOMENTUM).minimize(cost, global_step=global_step)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)


    #ctc_ greedy_decoder
    decoded, log_prob = tf.nn.ctc_beam_search_decoder(logits, seq_len, merge_repeated=False)

    acc = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), targets))

    init = tf.global_variables_initializer()

    def report_accuracy(decoded_list, test_targets):
        original_list = decode_sparse_tensor(test_targets)
        detected_list = decode_sparse_tensor(decoded_list)
        true_numer = 0

        if len(original_list) != len(detected_list):
            print("len(original_list)", len(original_list), "len(detected_list)", len(detected_list),
                  " test and detect length desn't match")
            return
        print("T/F: original(length) <-------> detectcted(length)")
        for idx, number in enumerate(original_list):
            detect_number = detected_list[idx]
            hit = (number == detect_number)
            print(hit, number, "(", len(number), ") <-------> ", detect_number, "(", len(detect_number), ")")
            if hit:
                true_numer = true_numer + 1
        print("Test Accuracy:", true_numer * 1.0 / len(original_list))

    def do_report(val_gen,num):
        for i in range(num):
            test_inputs, test_targets, test_seq_len = val_gen.next_batch()
            test_feed = {inputs: test_inputs,
                        targets: test_targets,
                        seq_len: test_seq_len}
            st =time.time()
            dd= session.run(decoded[0], test_feed)
            tim = time.time() -st
            print('time:%s'%tim)
            report_accuracy(dd, test_targets)

    def test_report(testi,files):
        true_numer = 0
        num = files//BATCH_SIZE

        for i in range(num):
            test_inputs, test_targets, test_seq_len = val_gen.next_batch()
            test_feed = {inputs: test_inputs,
                        targets: test_targets,
                        seq_len: test_seq_len}
            dd = session.run([decoded[0]], test_feed)
            original_list = decode_sparse_tensor(test_targets)
            detected_list = decode_sparse_tensor(dd)
            for idx, number in enumerate(original_list):
                detect_number = detected_list[idx]
                hit = (number == detect_number)
                if hit:
                    true_numer = true_numer + 1
        print("Test Accuracy:", true_numer * 1.0 / files)


    def do_batch(train_gen,val_gen):
        train_inputs, train_targets, train_seq_len = train_gen.next_batch()

        feed = {inputs: train_inputs, targets: train_targets, seq_len: train_seq_len}

        b_loss, b_targets, b_logits, b_seq_len, b_cost, steps, _ = session.run(
            [loss, targets, logits, seq_len, cost, global_step, optimizer], feed)

        #print(b_cost, steps)
        if steps > 0 and steps % REPORT_STEPS == 0:
            do_report(val_gen,test_num)
            saver.save(session, "./model/LPRtf3.ckpt", global_step=steps)
        return b_cost, steps

    with tf.Session() as session:
        session.run(init)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=100)
        if a=='train':
            for curr_epoch in range(num_epochs):
                print("Epoch.......", curr_epoch)
                train_cost = train_ler = 0
                for batch in range(BATCHES):
                    start = time.time()
                    c, steps = do_batch(train_gen,val_gen)
                    train_cost += c * BATCH_SIZE
                    seconds = time.time() - start
                    #print("Step:", steps, ", batch seconds:", seconds)

                train_cost /= TRAIN_SIZE
                val_cs=0
                val_ls =0
                for i in range(test_num):
                    train_inputs, train_targets, train_seq_len = val_gen.next_batch()
                    val_feed = {inputs: train_inputs,
                                targets: train_targets,
                                seq_len: train_seq_len}

                    val_cost, val_ler, lr, steps = session.run([cost, acc, learning_rate, global_step], feed_dict=val_feed)
                    val_cs+=val_cost
                    val_ls+=val_ler

                log = "Epoch {}/{}, steps = {}, train_cost = {:.3f}, train_ler = {:.3f}, val_cost = {:.3f}, val_ler = {:.3f}, time = {:.3f}s, learning_rate = {}"
                print(log.format(curr_epoch + 1, num_epochs, steps, train_cost, train_ler, val_cs/test_num, val_ls/test_num,
                                 time.time() - start, lr))
        if a =='test':
            testi='valid'
            saver.restore(session, './model8.24best/LPRtf3.ckpt-25000')
            test_gen = TextImageGenerator(img_dir=testi,
                                           label_file=None,
                                           batch_size=BATCH_SIZE,
                                           img_size=img_size,
                                           num_channels=num_channels,
                                           label_len=label_len)
            do_report(test_gen,3)

In [0]:
a = input('train or test:')
train(a)

In [0]:
!unzip synthetic-turkish-license-plates.zip

In [0]:
!ls

In [0]:
# imports 
import pandas as pd
import numpy as np
import glob
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

from keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras.utils import Sequence, plot_model
# determining file path 
files = glob.glob("synthetic-turkish-license-plates/license-plates/*.png")
all_files = [(os.path.splitext(os.path.basename(file))[0], file) for file in files]
#determining the images and the Labels and reomve the dash.
X = [filename[1] for filename in all_files]
y1 = np.array([plate_label[0].replace("-", "")[0] for plate_label in all_files])
y2 = np.array([plate_label[0].replace("-", "")[1] for plate_label in all_files])
y3 = np.array([plate_label[0].replace("-", "")[2] for plate_label in all_files])
y4 = np.array([plate_label[0].replace("-", "")[3] for plate_label in all_files])
y5 = np.array([plate_label[0].replace("-", "")[4] for plate_label in all_files])
y6 = np.array([plate_label[0].replace("-", "")[5] for plate_label in all_files])
y7 = np.array([plate_label[0].replace("-", "")[6] for plate_label in all_files])
# shuffle the data. 
X, y1, y2, y3, y4, y5, y6, y7 = shuffle(X, y1, y2, y3, y4, y5, y6, y7)
categories = np.array(list("ABCDEFGHIJKLMNOPRSTUVYZ0123456789"))
onehot_enc = OneHotEncoder(sparse=False)
onehot_enc.fit(categories.reshape(-1, 1))

y1 = onehot_enc.transform(y1.reshape(-1, 1))
y2 = onehot_enc.transform(y2.reshape(-1, 1))
y3 = onehot_enc.transform(y3.reshape(-1, 1))
y4 = onehot_enc.transform(y4.reshape(-1, 1))
y5 = onehot_enc.transform(y5.reshape(-1, 1))
y6 = onehot_enc.transform(y6.reshape(-1, 1))
y7 = onehot_enc.transform(y7.reshape(-1, 1))

X_train, X_test, \
y1_train, y1_test, \
y2_train, y2_test, \
y3_train, y3_test, \
y4_train, y4_test, \
y5_train, y5_test, \
y6_train, y6_test, \
y7_train, y7_test = \
    train_test_split(X, y1, y2, y3, y4, y5, y6, y7, test_size=0.2, random_state=42)

class MyCustomGenerator(Sequence):
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y1 = self.labels[0][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y2 = self.labels[1][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y3 = self.labels[2][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y4 = self.labels[3][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y5 = self.labels[4][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y6 = self.labels[5][idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y7 = self.labels[6][idx * self.batch_size : (idx+1) * self.batch_size]
        
        return np.array([cv2.imread(filename) / 255.0
                         for filename in batch_x]), \
            [np.array(batch_y1), np.array(batch_y2), np.array(batch_y3), np.array(batch_y4), np.array(batch_y5), np.array(batch_y6), np.array(batch_y7)]

batch_size = 200

training_batch_generator = MyCustomGenerator(X_train, (y1_train, y2_train, y3_train, y4_train, y5_train, y6_train, y7_train), batch_size)
test_batch_generator = MyCustomGenerator(X_test, (y1_test, y2_test, y3_test, y4_test, y5_test, y6_test, y7_test), batch_size)

WIDTH = 1025
HEIGHT = 218
CHANNEL = 3

inputs = Input(shape=(HEIGHT, WIDTH, CHANNEL))
conv1 = Conv2D(8, kernel_size=3, activation="relu")(inputs)
conv1 = Conv2D(8, kernel_size=3, activation="relu")(conv1)
conv1 = MaxPooling2D()(conv1)

conv2 = Conv2D(16, kernel_size=3, activation="relu")(conv1)
conv2 = Conv2D(16, kernel_size=3, activation="relu")(conv2)
conv2 = MaxPooling2D()(conv2)

conv3 = Conv2D(16, kernel_size=3, activation="relu")(conv2)
conv3 = Conv2D(16, kernel_size=3, activation="relu")(conv3)
conv3 = MaxPooling2D()(conv3)

dense = Flatten()(conv3)

dense = Dense(64, activation="relu")(dense)

dense_output1 = Dense(33, activation="softmax")(dense)
dense_output2 = Dense(33, activation="softmax")(dense)
dense_output3 = Dense(33, activation="softmax")(dense)
dense_output4 = Dense(33, activation="softmax")(dense)
dense_output5 = Dense(33, activation="softmax")(dense)
dense_output6 = Dense(33, activation="softmax")(dense)
dense_output7 = Dense(33, activation="softmax")(dense)

model = Model(inputs=inputs, \
              outputs=[dense_output1, dense_output2, dense_output3, dense_output4, dense_output5, dense_output6, dense_output7])
model.summary()

plot_model(model)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit_generator(training_batch_generator, epochs=32)

predictions = model.predict_generator(test_batch_generator)

y1_hat = onehot_enc.inverse_transform(predictions[0])
y2_hat = onehot_enc.inverse_transform(predictions[1])
y3_hat = onehot_enc.inverse_transform(predictions[2])
y4_hat = onehot_enc.inverse_transform(predictions[3])
y5_hat = onehot_enc.inverse_transform(predictions[4])
y6_hat = onehot_enc.inverse_transform(predictions[5])
y7_hat = onehot_enc.inverse_transform(predictions[6])

y1_test = onehot_enc.inverse_transform(y1_test)
y2_test = onehot_enc.inverse_transform(y2_test)
y3_test = onehot_enc.inverse_transform(y3_test)
y4_test = onehot_enc.inverse_transform(y4_test)
y5_test = onehot_enc.inverse_transform(y5_test)
y6_test = onehot_enc.inverse_transform(y6_test)
y7_test = onehot_enc.inverse_transform(y7_test)

print(classification_report(y1_test, y1_hat))
print("#" * 40)
print(classification_report(y2_test, y2_hat))
print("#" * 40)
print(classification_report(y3_test, y3_hat))
print("#" * 40)
print(classification_report(y4_test, y4_hat))
print("#" * 40)
print(classification_report(y5_test, y5_hat))
print("#" * 40)
print(classification_report(y6_test, y6_hat))
print("#" * 40)
print(classification_report(y7_test, y7_hat))
print("#" * 40)

test_image = next(iter(test_batch_generator))
print(test_image[0][1].shape)
plt.imshow(test_image[0][1])
plt.show()

prediction = model.predict(test_image[0][1].reshape(1, 218, 1025, 3))

for ch in prediction:
    print(onehot_enc.inverse_transform(ch)[0][0], end="")


In [0]:
model.save('BestModel.h5')

In [0]:
test_image = next(iter(test_batch_generator))
print(test_image[0][10].shape)
plt.imshow(test_image[0][10])
plt.show()

prediction = model.predict(test_image[0][10].reshape(1, 218, 1025, 3))

for ch in prediction:
    print(onehot_enc.inverse_transform(ch)[0][0], end="")
